In [1]:
transform_id = 1
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = SessionHelper().session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


2019-07-17 19:11:44,543 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-07-17 19:11:44,575 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-07-17 19:11:44,619 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-07-17 19:11:44,623 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-07-17 19:11:44,639 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-07-17 19:11:44,642 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-07-17 19:11:44,646 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-07-17 19:11:44,652 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-07-17 19:11:4

# CORE Cartridge Notebook::[Pending New Enrichments]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [2]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Include your input dataset(s) here. Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

In [3]:
transform = Transform()

transform.name = 'pending_new_enrich'
transform.brand = 'ofev'
transform.state = 'enrich'
transform.pharmaceutical_company = 'bi'
transform.filesystem_path = 's3://ichain-dev/movahlx/bi/02_status_mapping'

transform.secret_name = 'bi'
transform.secret_type_of = 'database'

In [4]:
import re

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from s3fs import S3FileSystem
#import mysql.connector as mysql

from core.secret import Secret

In [5]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead
trans_id = 'pmcTransactionId'
product = 'medication'
patient_id = 'pmcPatientId'
pharm = 'pharmName'
status_date = 'statusDate'
ref_date = 'refDate'
status =  'statusCode'
substatus =  'subStatus'
ic_status = 'integrichain_status'
ic_substatus = 'integrichain_sub_status'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This transform handles the situations where the first status is active or cancelled and the first status date received exceeds the one day grace period. These instances are handled by making the first status date also the first referral date and making this new row a pending new status. Will finish with more rows than started with.

### Transformation

In [6]:
def pandas_from_parquet_s3(file_path):
    
    s3 = S3FileSystem()
    df = (
        pq
        .ParquetDataset(file_path, filesystem=s3)
        .read_pandas()
        .to_pandas()
    )
    
    return df


def pandas_from_database(query, *args):
    
    connection = mysql.connect(
        host=secret.host, 
        user=secret.user, 
        passwd=secret.password, 
        port=secret.port, 
        charset='utf8'
    )
    
    query = query.format(*args)
    df = pd.read_sql(sql=query, con=connection)
    connection.close()
    
    return df


def pandas_to_parquet_s3(df, partition, file_path):
    
    s3 = S3FileSystem()
    table = pa.Table.from_pandas(df)
    
    pq.write_to_dataset(table, file_path, partition_cols=partition, filesystem=s3)

In [7]:
df = pandas_from_parquet_s3(transform.filesystem_path)
df.head()

,pharmName,sp_file_id,sp_file_dt,sp_file_tm,sp_rec_id,restmt_flg,pharmy_npi_id,ncpdp_id,pharmy_nm,statusCode,...,dx2Code,custStatusDescription,transferPatient,hubPatient,bi_status_code,bi_status_description,brand,integrichain_status,integrichain_sub_status,subStatus
0,BRIOVARX,811072016,20161107,080008,None,N,1083045140,1564930,"BRIOVARX OF INDIANA, LLC",ACTIVE,...,,,,,S01,PRODUCT SHIPPED TO PATIENT,OFEV,ACTIVE,SHIPMENT,SHIPMENT
1,BRIOVARX,811072016,20161107,080008,None,N,1083045140,1564930,"BRIOVARX OF INDIANA, LLC",ACTIVE,...,,,,,S01,PRODUCT SHIPPED TO PATIENT,OFEV,ACTIVE,SHIPMENT,SHIPMENT
2,BRIOVARX,811072016,20161107,080008,None,N,1083045140,1564930,"BRIOVARX OF INDIANA, LLC",ACTIVE,...,,,,,S01,PRODUCT SHIPPED TO PATIENT,OFEV,ACTIVE,SHIPMENT,SHIPMENT
3,BRIOVARX,811072016,20161107,080008,None,N,1083045140,1564930,"BRIOVARX OF INDIANA, LLC",ACTIVE,...,,,,,S01,PRODUCT SHIPPED TO PATIENT,OFEV,ACTIVE,SHIPMENT,SHIPMENT
4,BRIOVARX,811072016,20161107,080008,None,N,1083045140,1564930,"BRIOVARX OF INDIANA, LLC",ACTIVE,...,,,,,S01,PRODUCT SHIPPED TO PATIENT,OFEV,ACTIVE,SHIPMENT,SHIPMENT


In [19]:
def load_clean_file(cust_df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus):
    
    # Extract and map relevant columns
    #cust_df = cust_input_df.loc[:,[trans_id
    #                               patient_id,
    #                               pharm,
    #                               product,
    #                               status_date,
    #                               ref_date,
    #                               status,
    #                               substatus]]

    cust_df = cust_df.rename(columns={trans_id:'trans_id',
                                      patient_id:'patient_id',
                                      pharm:'pharm',
                                      product:'product',
                                      status_date:'status_date',
                                      ref_date:'ref_date',
                                      status:'status_code',
                                      substatus:'substatus_code'})
    
    # Convert dates to datetime format
    cust_df.status_date = pd.to_datetime(cust_df.status_date)
    cust_df.ref_date = pd.to_datetime(cust_df.ref_date)
    
    ## Extract brand from medication
    cust_df['product'] = cust_df['product'].apply(lambda x: x.split()[0].strip())
    
	## Convert status codes to uppercase
    cust_df.status_code = cust_df.status_code.str.upper()
    cust_df.substatus_code = cust_df.substatus_code.str.upper()
    
	## Fill missing referral_date with min(status_date)
	
    #min_status_dates=cust_df.groupby(['patient_id','pharm','product'])['status_date'].min().reset_index().rename(columns={'status_date':'min_status_date'})
    
    #cust_df = pd.merge(cust_df, min_status_dates, how='inner', on=['patient_id','pharm','product'])
    
    #cust_df.loc[cust_df['ref_date'].isnull(), ['ref_date']] = cust_df['min_status_date']
    
    #cust_df = cust_df.drop(['min_status_date'],axis=1).drop_duplicates()
    
    #cust_df.sort_values(by=['patient_id', 'pharm', 'product', 'status_date'], ascending=[True, True, True, True], inplace=True)
    #cust_df = cust_df.reset_index(drop=True)

    # Output clean customer file
    return cust_df

In [20]:
def pending_new_enrichment(df: pd.DataFrame, table_columns):
    status_date = 'status_date'
    ref_date = 'ref_date'
    patient = 'patient_id'
    pharm = 'pharm'
    product = 'product'
    substatus_code = 'substatus_code'
    ic_status = 'integrichain_status'
    ic_substatus = 'integrichain_sub_status'
    pending_status = 'PENDING'
    active_status = 'ACTIVE'
    cancelled_status = 'CANCELLED'
    pending_new_substatus = 'NEW'
    #statuses.pending_new_substatus
    status_code = 'status_code'

    df.sort_values(by=[patient, pharm, product, status_date], ascending=[True, True, True, True], inplace=True)

    min_date_df = df.groupby([patient, pharm, product])[status_date].min().reset_index().rename(
        columns={status_date: 'First_Status_Date'})
    df = pd.merge(df, min_date_df, how='inner', on=[patient, pharm, product])
    min_ref_date_df = df.groupby([patient, pharm, product])[ref_date].min().reset_index().rename(
        columns={ref_date: 'First_Ref_Date'})
    df = pd.merge(df, min_ref_date_df, how='inner', on=[patient, pharm, product])
    #df['Is_Pending'] = np.where(df[ic_status] == pending_status, 1, 0)
    #pending_df = df.groupby([patient, pharm, product])['Is_Pending'].sum().reset_index().rename(
    #    columns={'Is_Pending': 'Has_Pending'})
    #df = pd.merge(df, pending_df, how='inner', on=[patient, pharm, product])
    #df['Has_Pending'] = np.where(df['Has_Pending'] > 0, 1, 0)
    #df.drop(columns=['Is_Pending'], axis=1, inplace=True)
    df.sort_values(by=[patient, pharm, product, status_date], ascending=[True, True, True, True], inplace=True)
    first_status = df.groupby([pharm, product, patient])[ic_status].first().reset_index().rename(
        columns={ic_status: 'First_IC_Status'})
    first_status['First_Status_Active_Cancelled'] = np.where(
        first_status['First_IC_Status'].isin([active_status, cancelled_status]), 1, 0)
    df = pd.merge(df, first_status, how='inner', on=[pharm, product, patient])
    pat_df = df.drop_duplicates(subset=[patient, pharm, product], keep='first')
    pat_df['Day_Diff'] = (pat_df['First_Status_Date'] - pat_df['First_Ref_Date']) / np.timedelta64(1, 'D')
    pat_df = pat_df[[patient, pharm, product, 'Day_Diff']]
    df = pd.merge(df, pat_df, how='inner', on=[pharm, product, patient])
    to_enrich_df = df[((df['First_Status_Active_Cancelled'] == 1) 
                      # & (df['Has_Pending'] == 0) 
                      & (df['Day_Diff'] > 1))]
    to_enrich_df = to_enrich_df.drop_duplicates(subset=[patient, pharm, product])
    
    to_enrich_df.loc[:, substatus_code] = pending_new_substatus
    to_enrich_df.loc[:, status_code] = pending_status
    to_enrich_df.loc[:, ic_substatus] = pending_new_substatus
    to_enrich_df.loc[:, ic_status] = pending_status
    to_enrich_df.loc[:, 'pjh'] = 'No Status Clarity'
    to_enrich_df.loc[:, status_date] = to_enrich_df['First_Ref_Date']
    to_enrich_df.loc[:, ref_date] = to_enrich_df['First_Ref_Date']
    to_enrich_df.loc[:, 'trans_id'] = 'ic_' + to_enrich_df['trans_id'].astype(str) + '_pn'
    to_enrich_df.loc[:, 'enrichment'] = 'pending new'
    #to_enrich_df = to_enrich_df[table_columns]
    df = df.append(to_enrich_df)
    #df = df[table_columns]
    return (df)

In [21]:
def file_standard(cust_df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus):

    cust_df = cust_df.rename(columns={'trans_id': trans_id,
                                      'patient_id': patient_id,
                                      'pharm': pharm,
                                      'product': product,
                                      'status_date': status_date,
                                      'ref_date': ref_date,
                                      'status_code': status,
                                      'substatus_code': substatus})
    return cust_df

In [22]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

## Dont forget the heirarchy column. Then qa this
df = load_clean_file(df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus)
columns = list(df.columns)
columns.append('pjh')
enriched = pending_new_enrichment(df, columns)
#final_dataframe = file_standard(enriched, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Day_Diff,First_IC_Status,First_Ref_Date,First_Status_Active_Cancelled,First_Status_Date,MP_CITY_NM,PresAddr1,Primary_PayerName,Secondary_PayerName,bi_status_code,...,sp_file_dt,sp_file_id,sp_file_tm,sp_pat_id,sp_rec_id,status_code,status_date,substatus_code,trans_id,transferPatient
0,1.0,ACTIVE,2015-08-25,1,2015-08-26,PRESCOTT,3777 CROSSING DRIVE,ARIZONA MEDICAID-MEDIMPACT,None,S01,...,20150831,108292015,073655,23243120,82615246,ACTIVE,2015-08-26,SHIPMENT,7517332,
1,1.0,ACTIVE,2015-08-25,1,2015-08-26,PRESCOTT,3777 CROSSING DRIVE,ARIZONA MEDICAID-MEDIMPACT,None,S01,...,20150921,109192015,213933,23243120,91715174,ACTIVE,2015-09-17,SHIPMENT,7832225,
2,1.0,ACTIVE,2015-08-25,1,2015-08-26,PRESCOTT,3777 CROSSING DRIVE,ARIZONA MEDICAID-MEDIMPACT,None,S01,...,20151026,110242015,060619,23243120,102215159,ACTIVE,2015-10-22,SHIPMENT,8370416,
3,1.0,ACTIVE,2015-08-25,1,2015-08-26,PRESCOTT,3777 CROSSING DRIVE,ARIZONA MEDICAID-MEDIMPACT,None,S01,...,20151123,111212015,063445,23243120,112015144,ACTIVE,2015-11-20,SHIPMENT,8915363,
4,1.0,ACTIVE,2015-08-25,1,2015-08-26,PRESCOTT,3777 CROSSING DRIVE,ARIZONA MEDICAID-MEDIMPACT,None,S01,...,20151228,112262015,063521,23243120,12221581,ACTIVE,2015-12-22,SHIPMENT,9479726,


In [23]:
len(df)

545419

In [24]:
len(enriched)

555214

In [26]:
worked =enriched[~(enriched['trans_id'].isin(df['trans_id']))]
worked.loc[:, ['First_Status_Date', 
               'First_Ref_Date', 
               'status_code', 
               'First_Status_Active_Cancelled', 
               'First_IC_Status', 
               'trans_id']]

,First_Status_Date,First_Ref_Date,status_code,First_Status_Active_Cancelled,First_IC_Status,trans_id
32,2015-08-25,2015-08-21,PENDING,1,CANCELLED,ic_7518205_pn
33,2018-12-02,2018-11-26,PENDING,1,ACTIVE,ic_36134092_pn
101,2015-07-03,2015-06-26,PENDING,1,ACTIVE,ic_6803725_pn
132,2015-06-29,2015-06-26,PENDING,1,CANCELLED,ic_6803646_pn
173,2015-07-20,2015-07-08,PENDING,1,CANCELLED,ic_7431837_pn
192,2015-07-02,2015-06-30,PENDING,1,CANCELLED,ic_6803636_pn
201,2015-07-07,2015-07-01,PENDING,1,CANCELLED,ic_7431665_pn
202,2015-06-30,2015-06-25,PENDING,1,CANCELLED,ic_6803648_pn
203,2015-07-30,2015-07-20,PENDING,1,ACTIVE,ic_7432073_pn
316,2015-07-02,2015-06-26,PENDING,1,ACTIVE,ic_6804044_pn


### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe)